# 1. Imports

In [4]:
import numpy as np
from math import pi

In [5]:
from tensorflow.keras.models import load_model

In [6]:
from tensorflow.keras.losses import MSE
from tensorflow.keras.losses import categorical_crossentropy as CCE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

import tensorflow as tf

In [7]:
# Load properties from another python file
from properties import *

In [8]:
from random import random, randint

In [9]:
path_model = "ACAS_XU_tf_keras/ACASXU_1_1.h5"

# 2. Load models

In [10]:
model_11 = load_model(path_model)

In [ ]:
model_35 = load_model("ACAS_XU_tf_keras/ACASXU_3_5.h5")

In [11]:
model_11.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_98 (Dense)            (None, 50)                300       
                                                                 
 activation_98 (Activation)  (None, 50)                0         
                                                                 
 dense_99 (Dense)            (None, 50)                2550      
                                                                 
 activation_99 (Activation)  (None, 50)                0         
                                                                 
 dense_100 (Dense)           (None, 50)                2550      
                                                                 
 activation_100 (Activation)  (None, 50)               0         
                                                                 
 dense_101 (Dense)           (None, 50)              

In [12]:
model_11.compile()

In [ ]:
model_35.compile()

In [ ]:
pt = np.array([1.0, 1.0, 1.0, 1.0, 1.0]).reshape(1,5)
model_11.predict(pt)

In [ ]:
pt = np.array([1.0, 1.0, 1.0, 1.0, 1.0]).reshape(1,5)
model_35.predict(pt)

# 3. FGSM

In [ ]:
def generate_adv_sample(model, x0, label, loss_function="MSE", eps=1e-5):
    # transforming into a tensorflow object
    x0_ = tf.cast(x0, tf.float32)
    
    # record our gradients
    with tf.GradientTape() as tape:
        # explicitly indicate that our input should be tacked for gradient updates
        tape.watch(x0_)

        # use our model to make predictions on the input and then compute the loss
        pred = model(x0_)
        if loss_function == "CCE":
            np_label = np.array([i==label for i in range(0,5)]).reshape((1,5))
            loss = CCE(np_label, pred)
        elif loss_function == "MSE":
            loss = MSE(label, pred)
        else:
            raise Exception("Unknown loss function '{0}'".format(loss_function))
        
        # calculate the gradients of loss with respect to the input, then compute the sign of the gradient
        gradient = tape.gradient(loss, x0_)
        signedGrad = tf.sign(gradient)

        # construct the image adversary
        adv_sample = (x0_ + (signedGrad * eps)).numpy()

        # return the adversarial sample to the calling function
        return(adv_sample)

In [ ]:
x0_adv = generate_adv_sample(model_11, pt, label = 0, loss_function = "CCE", eps=1e-3)
print(x0_adv)

In [ ]:
model_11.predict(x0_adv)

In [ ]:
np_label = np.array([int(i==2) for i in range(0,5)]).reshape((1,5))
np_label

# 4. Random search

In [13]:
def random_pts(n, prop_dom, main_dom):
    """ prop_dom is a list of domains given by intervals in a numpy 2x5 table. """
    ndom = len(prop_dom)
    ndim = main_dom.shape[1]
    
    x = np.zeros((n,ndim))
    for i in range(n): # generate the i-th point
        choosen_dom = prop_dom[randint(0,ndom-1)] # choose the input property domain for a given prop
        for k in range(ndim): # create a random coord for each dim
            boundaries = main_dom[:,k]
            if choosen_dom[0,k] != None:
                boundaries[0] = choosen_dom[0,k]
            if choosen_dom[1,k] != None:
                boundaries[1] = choosen_dom[1,k]
            x[i,k] = boundaries[0] + random()*(boundaries[1]-boundaries[0])

    return(x)

In [14]:
def normalize(x, x_mean, x_range):
    (n,k) = x.shape
    nx = np.zeros((n,k))
    for i in range(n):
        for j in range(k):
            nx[i,j] = (x[i,j]-x_mean[j])/x_range[j]
    return(nx)

In [ ]:
def check_pts(model, input_pts, IP, OP):
    n = input_pts.shape[0]
    IO_check = np.zeros((n,2)) # 1st column : input checked - 2nd column : output checked
    
    norm_input = normalize(input_pts, X_mean, X_range) # normalize pts
    
    pred_pts = model.predict(norm_input) # make predictions with the model (neural net)
    
    for k in range(n):
        IO_check[k,0] = IP(input_pts[k,:]) # check input (just in case)
        IO_check[k,1] = OP(pred_pts[k,:]) # check output

    return(IO_check, pred_pts)

In [23]:
rand_inputs = random_pts(10, IP2_dom, X_dom)
print(rand_inputs)

[[ 5.65399259e+04 -1.06280337e+00  6.93948977e-01  1.14894048e+03
   5.46869882e+01]
 [ 5.62140790e+04  1.82724881e+00  2.81804609e+00  1.18205705e+03
   9.19801608e-01]
 [ 5.83717144e+04  1.28881663e+00  5.32541245e-01  1.19100463e+03
   4.37645376e+01]
 [ 5.88605325e+04 -2.55605764e-01 -2.53960890e+00  1.19430519e+03
   5.80021110e+00]
 [ 5.72159842e+04 -5.61606981e-01 -1.59989365e+00  1.15520853e+03
   2.95549577e+01]
 [ 5.85248429e+04  2.03645450e+00 -1.58755871e+00  1.18583048e+03
   5.86414730e+01]
 [ 5.63006282e+04 -8.60885938e-01  1.45061203e+00  1.19775165e+03
   1.98538536e+01]
 [ 5.88870929e+04  2.13785846e+00 -1.34665514e+00  1.17127115e+03
   3.36253026e+01]
 [ 5.63375468e+04 -2.39428233e+00  2.42154936e-01  1.15041568e+03
   1.00649320e+01]
 [ 5.90527600e+04 -1.34307909e+00 -2.07606938e+00  1.19029353e+03
   2.10400198e+01]]


In [ ]:
Pcheck, pred_pts = check_pts(model_11, rand_inputs, IP1, OP1)

In [ ]:
def find_adverse(input_pts, prop_check):
    n = prop_check.shape[0]
    index = []
    for k in range(n):
        if prop_check[k,0] and not(prop_check[k,1]):
            index.append(k)
    return(input_pts[index,:])

In [ ]:
adv = find_adverse(rand_inputs, Pcheck)
print(adv.shape)

(0, 5)


In [ ]:
Pcheck.shape

(1000000, 2)

# 5. FGSM

In [25]:
def bool_to_binary(b):
    if b:
        return 1
    return 0

def fgsm(x,model,var,label):

    x = tf.cast(x, tf.float32)
    eps=[(X_dom[1][i]-X_dom[0][i])*var for i in range(5)]
    # record our gradients
    with tf.GradientTape() as tape:
        # explicitly indicate that our image should be tacked for gradient updates
        tape.watch(x)

        # use our model to make predictions on the input image and then compute the loss
        #label = model.predict(x)
        loss = MSE(label, [0,0,0,0,0])
        
        # calculate the gradients of loss with respect to the image, then compute the sign of the gradient
        gradient = tape.gradient(loss, x)
        signedGrad = tf.sign(gradient)

        # construct the image adversary
        adversary = (x + (signedGrad * eps))
        adversary_img=model.predict(adversary)
        
        argmin_label= np.argmin(label)
        argmin_adversary=np.argmin(adversary_img)

        same= (argmin_label==argmin_adversary)
        same=bool_to_binary(same)

        return(same,argmin_label,argmin_adversary)

def stats_fgsm(model,dataset,var):
    label=model.predict(dataset)
    res=np.zeros((5,5))
    for i in range(len(dataset)):
        same,argmin_label,argmin_adversary=fgsm(dataset[i],model,var,label[i])
        res[argmin_label][argmin_adversary]+=1
    return res

rand_inputs = random_pts(10000, IP1_dom, X_dom)
rand_inputs = normalize(rand_inputs, X_mean, X_range)
stats=stats_fgsm(model_11, rand_inputs,0.01)

    

ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.